#imports

In [1]:
import nltk
nltk.download('punkt')
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Colab Notebooks/ece1786-project

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Mounted at /content/drive
/content/drive/MyDrive/Colab Notebooks/ece1786-project


In [2]:
import torch 
import numpy as np

from nltk.tokenize import sent_tokenize 

from pathlib import Path 
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset
from torch.utils.data.dataloader import DataLoader
from mingpt.bpe import BPETokenizer 
from mingpt.utils import set_seed 
from mingpt.model import GPT
import pandas as pd
set_seed(1234)

In [3]:
!pip install evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 72 kB 1.1 MB/s 
     |████████████████████████████████| 115 kB 11.0 MB/s 
     |████████████████████████████████| 451 kB 56.0 MB/s 
     |████████████████████████████████| 212 kB 43.0 MB/s 
     |████████████████████████████████| 182 kB 56.8 MB/s 
     |████████████████████████████████| 127 kB 46.5 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [4]:
import evaluate

In [5]:
!pip install jiwer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.4 MB 5.1 MB/s 
     |████████████████████████████████| 2.2 MB 33.2 MB/s 


In [6]:
!pip install transformers
from transformers import AutoModelForCausalLM, AutoTokenizer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 5.2 MB/s 
     |████████████████████████████████| 7.6 MB 31.5 MB/s 


# Load model and eval

In [7]:
device = torch.device('cpu')

In [8]:
model = AutoModelForCausalLM.from_pretrained("./gpt2model").to(device)

In [9]:
tokenizer = AutoTokenizer.from_pretrained("gpt2", use_fast=True)

Downloading:   0%|          | 0.00/665 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [10]:
def score(metric="bleu", filename="cocktail_dataset.pkl"):

  def create_propmts():
    df = pd.read_pickle(filename)
    answers = []
    prompts = []
    for i in range(len(df.index)):
      recipe = "RECIPE NAME\n" + df.loc[i, "Name"] + " \n\nRECIPE INGREDIENTS\n"
      prompts.append(recipe)
      for ingredient in df.loc[i, "Ingredients"]:
        recipe += ingredient + "\n"
      recipe += "\nRECIPE INSTRUCTIONS\n" 
      for instruction in  df.loc[i, "Instructions"]:
        recipe += instruction + "\n"
      if metric == "bleu":
        answers.append([recipe])
      else:
        answers.append(recipe)
    return answers, prompts

  answers, prompts = create_propmts()
  attempts = []
  for i in range(len(answers)):
    encoded_prompt = tokenizer(prompts[i], return_tensors="pt").input_ids.to(device)
    generated_sequence = model.generate(encoded_prompt, do_sample=True, max_length=120, temperature = 0.7, return_dict_in_generate=True, output_scores=True)
    # tokenizer.batch_decode(generated_sequence.sequences, skip_special_tokens=True)[0]
    attempts.append(tokenizer.batch_decode(generated_sequence.sequences, skip_special_tokens=True)[0])
  
  eval = evaluate.load(metric)
  results = eval.compute(predictions=attempts, references=answers)
  return results

In [11]:
score(metric="wer")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generati

0.8138725549021961

In [12]:
score(metric="bleu")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generati

{'bleu': 0.17047678855598955,
 'precisions': [0.5838129167808532,
  0.34276714993746815,
  0.22829355384326255,
  0.1611134944468278],
 'brevity_penalty': 0.5820236615536167,
 'length_ratio': 0.6488264580369844,
 'translation_length': 21894,
 'reference_length': 33744}

## examples


In [19]:
prompt = "RECIPE NAME\nWhiskey Sour"
input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device)

outputs = model.generate(input_ids, do_sample=True, max_length=128,temperature = 0.7, return_dict_in_generate=True, output_scores=True)
print(tokenizer.batch_decode(outputs.sequences, skip_special_tokens=True)[0])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


RECIPE NAME
Whiskey Sour 

RECIPE INGREDIENTS
3/4 ounce bourbon
3/4 ounce lime juice, freshly squeezed
1/2 ounce simple syrup
Garnish: lemon twist

RECIPE INSTRUCTIONS
Add bourbon, lime juice, simple syrup and ice into a shaker with ice and shake until well-chilled.
Strain into a rocks glass over crushed ice.
Garnish with a lemon twist.

